In [1]:
modello_per_analisi = 'Opel_Corsa'

### 0. Import funzioni e librerie

In [2]:
import os
import sys
from pathlib import Path
import importlib
import requests
import pandas as pd
import joblib

import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

In [3]:
source_root = Path.cwd().parents[0]
# Percorsi delle cartelle contenenti i moduli
dashboard_dir = source_root / '4_Dashboard' / 'Utils'

# Aggiungi al sys.path
sys.path.extend([str(dashboard_dir)])

# Import
from Functions_dashboard import get_data_dummy_for_dash, create_dashboard

In [4]:
# --- IMPORT CONFIGURAZIONE DELL'AUTO ---
# Percorsi delle cartelle contenenti i moduli
config_path = Path.cwd().parents[1] / 'Data'/ 'config'
# Aggiungi al sys.path
sys.path.extend([str(config_path)])
print('Il config_dir risulta:', config_path, ',')
print("il cui contenuto della cartella:", os.listdir(config_path))
config_path
# --- Import dinamico del modulo di configurazione in base al modello ---
modulo_config = f"config_{modello_per_analisi}"  # esempio: config_Mercedes_Classe_A
try:
    config = importlib.import_module(modulo_config)
    print(f"✅ Modulo di configurazione '{modulo_config}' caricato.")
except ModuleNotFoundError:
    print(f"❌ Modulo di configurazione '{modulo_config}' non trovato.")
    sys.exit(1)
# --- Accesso ai parametri di configurazione ---
#allestimenti = config.allestimenti
allestimento_performance = config.allestimento_performance
allestimento_sport = config.allestimento_sport
allestimento_middle = config.allestimento_middle
allestimento_base = config.allestimento_base
motorizzazioni_norm = config.motorizzazioni_norm
modelli_ord = config.modelli_ord
mappa_cv = config.mappa_cv
mappa_allestimenti = config.mappa_allestimenti

Il config_dir risulta: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/config ,
il cui contenuto della cartella: ['config_Audi_A3.py', 'config_Opel_Corsa.py', '__pycache__', 'config_BMW_Serie_1.py', 'config_Mercedes_Classe_A.py']
✅ Modulo di configurazione 'config_Opel_Corsa' caricato.


### 1. Dati

In [7]:
project_root = Path.cwd().parents[1]

model_ml = joblib.load(project_root / 'Data' / 'Modelli_ML' / modello_per_analisi / 'modello_rf.pkl')
scaler_ml = joblib.load(project_root / 'Data' / 'Modelli_ML' / modello_per_analisi / 'scaler_rf.pkl')
mse = joblib.load(project_root / 'Data' / 'Modelli_ML' / modello_per_analisi / 'mse_rf.pkl')
rmse = joblib.load(project_root / 'Data' / 'Modelli_ML' / modello_per_analisi / 'rmse_rf.pkl')
r2 = joblib.load(project_root / 'Data' / 'Modelli_ML' / modello_per_analisi / 'r2_rf.pkl')
cols_model = joblib.load(project_root / 'Data' / 'Modelli_ML' / modello_per_analisi / 'cols_model.pkl')

#### CARICAMENTO DATI SINGOLI
data_dummy = pd.read_csv(project_root / 'Data' / 'Processed_data' / modello_per_analisi / f'data_dummy_{modello_per_analisi}.csv')
data_originale = pd.read_csv(project_root / 'Data' / 'Processed_data' / modello_per_analisi / f'data_{modello_per_analisi}.csv')

In [8]:
allestimenti = list(set(allestimento_performance + allestimento_sport + allestimento_middle + allestimento_base))
carburanti = data_originale['Carburante'].unique().tolist()
val_metrics = dict(mse=mse,rmse=rmse,r2=r2)

cols_to_remove = ['prezzo_previsto', 'delta_prezzo', 'is_conveniente']
cols_model = [col for col in cols_model if col not in cols_to_remove]

### 2. Dashboard di previsione

In [ ]:
# ===============================
# Esempio di utilizzo
# ===============================
app = create_dashboard(modello_per_analisi, data_originale, allestimenti, carburanti, allestimento_performance,
                       allestimento_sport, allestimento_middle, model_ml, scaler_ml, val_metrics, cols_model, get_data_dummy_for_dash)

app.run(debug=True, port=8051) #port da modificare se necessario, se già in uso 